In [ ]:
# Model selection
MODEL_NAME = "maryasov/qwen2.5-coder-cline:7b-instruct-q8_0"
%env OLLAMA_CONTEXT_LENGTH=8192
%env OLLAMA_HOST=0.0.0.0
%env OLLAMA_KEEP_ALIVE=-1


In [ ]:
!apt-get install -y lshw pciutils
!nvcc --version
!nvidia-smi

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f"\n🧠 Available RAM: {ram_gb:.1f} GB")
print("✅ High-RAM runtime!" if ram_gb >= 20 else "❌ Not a high-RAM runtime.")


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh


In [ ]:
import subprocess
import time
import requests
import threading

# Start ollama serve in a background thread
def start_ollama():
    subprocess.call(['ollama', 'serve'])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.daemon = True
ollama_thread.start()

# Pull model (this also verifies Ollama CLI is ready)
!ollama pull {MODEL_NAME}

# Wait for Ollama HTTP API to be ready
def wait_for_ollama(timeout=60):
    for i in range(timeout):
        try:
            r = requests.get("http://localhost:11434")
            if r.status_code in [200, 404]:
                print(f"✅ Ollama is up (after {i+1}s).")
                return
        except requests.exceptions.ConnectionError:
            pass
        print(f"⏳ Waiting for Ollama to start... {i+1}s")
        time.sleep(1)
    raise RuntimeError("❌ Ollama did not start in time.")

wait_for_ollama()


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


In [ ]:
import re

# Run cloudflared tunnel in background and get the public URL
cloudflared_proc = subprocess.Popen(
    ['./cloudflared', 'tunnel', '--url', 'http://localhost:11434', '--no-autoupdate'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

public_url = None
for line in cloudflared_proc.stdout:
    print(line.strip())
    match = re.search(r'(https://.*\.trycloudflare\.com)', line)
    if match:
        public_url = match.group(1)
        break

if public_url:
    print(f"\n✅ Public URL for Ollama:\n{public_url}")
else:
    raise RuntimeError("❌ Could not find public Cloudflare URL.")


In [ ]:
import json

data = {
    "model": MODEL_NAME,
    "prompt": "Question: What is the capital of Japan?\nAnswer:",
    "stream": False
}

response = requests.post(f"{public_url}/api/generate", json=data)
print(response.json())
